In [1]:
import pandas as pd
import psycopg2
import time
import numpy as np
import re
from datetime import timedelta
import pickle
import matplotlib.pyplot as plt
from scipy.signal import butter, filtfilt
from scipy import stats
import neurokit2 as nk

In [2]:
pd.set_option('display.max_columns', None)

In [4]:
df = pd.read_csv("Descartes Data.csv")
df

,subject_id,block_type,q2,q3,q4,q6,q9,q10,tab_key,session_id,total_damage_taken,total_shot_all,total_kill,total_death,total_shot_hit,total_engage,rocket_shot_hit,gun_shot_hit,self_kill,gender,race,trait_score,state_score,trust_score,mode_cluster
0,BVCX,block_01,4,2,3,3,38,32,0,1263e8ef-4b1f-9a88-a07b-85971cac44ab,729.192802,141,2,11,14,17,0,14,0,Male,Prefer not to say,44.0,48.0,94,3
1,BVCX,block_02,4,4,3,4,21,18,0,6ee85647-4297-186c-24fa-c4970441459c,1583.916572,99,3,21,30,18,0,30,0,Male,Prefer not to say,44.0,48.0,94,3
2,BVCX,block_03,4,2,3,4,15,23,0,b8103d26-48b2-edb7-9cf6-febff2e0b27b,1035.726212,173,2,16,35,29,0,35,0,Male,Prefer not to say,44.0,48.0,94,3
3,BVCX,block_04,4,3,2,3,19,26,0,7dcb719f-46a8-2478-bac5-4da70d64241f,1045.448052,60,4,13,11,13,5,6,0,Male,Prefer not to say,44.0,48.0,94,3
4,CVBN,block_01,1,5,10,10,90,50,0,81378b98-41c5-d358-1442-72be5ced37bb,249.232862,138,9,3,39,16,0,39,0,Male,White/Caucasian,51.0,34.0,96,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
191,ZXWE,block_04,4,6,4,10,93,89,28,f916980f-4c87-8c5d-4e73-9e8eb6e76bc1,712.908842,162,18,7,69,28,7,62,0,Male,Asian,63.0,29.0,119,1
192,ZYXW,block_01,4,15,14,15,36,60,7,9d7674db-4299-49f2-09ce-408ffebe1332,856.348993,96,5,10,32,21,10,22,0,Male,White/Caucasian,NaN,NaN,94,1
193,ZYXW,block_02,4,8,19,6,20,32,13,ff9c203a-415a-f596-d37a-289a55618e1a,1200.748723,64,2,8,18,25,12,6,0,Male,White/Caucasian,NaN,NaN,94,1
194,ZYXW,block_03,4,12,10,13,15,23,8,da70cd6f-44d0-f518-f635-5fa866148670,1003.597517,97,4,4,27,27,8,19,0,Male,White/Caucasian,NaN,NaN,94,1


In [5]:
DB_NAME = "cognitive_battery"

conn = psycopg2.connect(
    host="127.0.0.1",
    port="5432",
    database=DB_NAME,
    user="postgres",
    password="postgres"
)



In [6]:
# Execute the SQL query
cursor = conn.cursor()
sql_query = "SELECT * FROM public.survey_data"
cursor.execute(sql_query)

# Fetch all the results
columns = [desc[0] for desc in cursor.description]
data = cursor.fetchall()
df_survey = pd.DataFrame(data, columns=columns)

cursor.close()
df_survey

,userid,enumeration_data,task_switching_data,working_memory_data,demographic_inventory_data,trust_scale_data,gaming_experience_data,state_trait_data,new_statetrait_data
0,HPLX,"{'BlockNo': [1, 2, 3], 'Q2': [10, 13, 15], 'Q3...","{'BlockNo': [1, 2, 3], 'Q2': [6, 14, 10], 'Q3'...","{'BlockNo': [1, 2, 3], 'Q2': [3, 13, 10], 'Q3'...","{'BlockNo': [1], 'Q2': [19], 'Q3': ['Male'], '...","{'BlockNo': [1], 'Q2': [3], 'Q3': [5], 'Q4': [...","{'BlockNo': [1], 'Q2': ['5-10 years'], 'Q3': [...",No Data,"{'BlockNo': [1], 'Q2': [4], 'Q3': [2], 'Q4': [..."
1,TYDF,"{'BlockNo': [1, 2, 3], 'Q2': [15, 6, 19], 'Q3'...","{'BlockNo': [1, 2], 'Q2': [3, 17], 'Q3': [19, ...","{'BlockNo': [1, 2, 3], 'Q2': [1, 2, 10], 'Q3':...","{'BlockNo': [1], 'Q2': [20], 'Q3': ['Male'], '...","{'BlockNo': [1], 'Q2': [5], 'Q3': [5], 'Q4': [...","{'BlockNo': [1], 'Q2': ['10 or more years'], '...","{'BlockNo': [1], 'Q2': [3], 'Q3': [3], 'Q4': [...",No Data
2,JFDE,"{'BlockNo': [1, 2, 3], 'Q2': [11, 12, 7], 'Q3'...","{'BlockNo': [1, 2, 3], 'Q2': [13, 12, 9], 'Q3'...","{'BlockNo': [1, 2, 3], 'Q2': [3, 8, 14], 'Q3':...","{'BlockNo': [1], 'Q2': [22], 'Q3': ['Male'], '...","{'BlockNo': [1], 'Q2': [5], 'Q3': [6], 'Q4': [...","{'BlockNo': [1], 'Q2': ['10 or more years'], '...",No Data,"{'BlockNo': [1], 'Q2': [4], 'Q3': [2], 'Q4': [..."
3,UJNH,"{'BlockNo': [1, 2, 3], 'Q2': [4, 10, 11], 'Q3'...","{'BlockNo': [1, 2, 3], 'Q2': [8, 4, 8], 'Q3': ...","{'BlockNo': [1, 2, 3], 'Q2': [1, 1, 11], 'Q3':...","{'BlockNo': [1], 'Q2': [20], 'Q3': ['Male'], '...","{'BlockNo': [1], 'Q2': [4], 'Q3': [6], 'Q4': [...","{'BlockNo': [1], 'Q2': ['10 or more years'], '...",No Data,"{'BlockNo': [1], 'Q2': [4], 'Q3': [1], 'Q4': [..."
4,MNBW,"{'BlockNo': [1, 2, 3], 'Q2': [20, 20, 10], 'Q3...","{'BlockNo': [1, 2, 3], 'Q2': [16, 8, 17], 'Q3'...","{'BlockNo': [1, 2, 3], 'Q2': [7, 16, 2], 'Q3':...","{'BlockNo': [1], 'Q2': [21], 'Q3': ['Male'], '...","{'BlockNo': [1], 'Q2': [5], 'Q3': [5], 'Q4': [...","{'BlockNo': [1], 'Q2': ['10 or more years'], '...","{'BlockNo': [1], 'Q2': [2], 'Q3': [3], 'Q4': [...",No Data
...,...,...,...,...,...,...,...,...,...
95,RIQW,"{'BlockNo': [1, 2, 3], 'Q2': [17, 19, 14], 'Q3...","{'BlockNo': [1, 2, 3], 'Q2': [15, 16, 9], 'Q3'...","{'BlockNo': [1, 2, 3], 'Q2': [11, 12, 5], 'Q3'...","{'BlockNo': [1], 'Q2': [18], 'Q3': ['Male'], '...","{'BlockNo': [1], 'Q2': [4], 'Q3': [6], 'Q4': [...","{'BlockNo': [1], 'Q2': ['10 or more years'], '...",No Data,"{'BlockNo': [1], 'Q2': [3], 'Q3': [3], 'Q4': [..."
96,LKPW,"{'BlockNo': [1, 2, 3], 'Q2': [8, 8, 15], 'Q3':...","{'BlockNo': [1, 2, 3], 'Q2': [10, 4, 5], 'Q3':...","{'BlockNo': [1, 2, 3], 'Q2': [12, 3, 13], 'Q3'...","{'BlockNo': [1], 'Q2': [20], 'Q3': ['Male'], '...","{'BlockNo': [1], 'Q2': [5], 'Q3': [4], 'Q4': [...","{'BlockNo': [1], 'Q2': ['10 or more years'], '...","{'BlockNo': [1], 'Q2': [3], 'Q3': [3], 'Q4': [...",No Data
97,VFTY,"{'BlockNo': [1, 2, 3], 'Q2': [16, 8, 20], 'Q3'...","{'BlockNo': [1, 2, 3], 'Q2': [12, 19, 10], 'Q3...","{'BlockNo': [1, 2, 3], 'Q2': [20, 13, 1], 'Q3'...","{'BlockNo': [1], 'Q2': [25], 'Q3': ['Male'], '...","{'BlockNo': [1], 'Q2': [7], 'Q3': [7], 'Q4': [...","{'BlockNo': [1], 'Q2': ['10 or more years'], '...","{'BlockNo': [1], 'Q2': [1], 'Q3': [2], 'Q4': [...",No Data
98,POIU,"{'BlockNo': [1, 2, 3], 'Q2': [19, 15, 20], 'Q3...","{'BlockNo': [1, 2, 3], 'Q2': [11, 6, 11], 'Q3'...","{'BlockNo': [1, 2, 3], 'Q2': [2, 15, 18], 'Q3'...","{'BlockNo': [1], 'Q2': [24], 'Q3': ['Male'], '...","{'BlockNo': [1], 'Q2': [5], 'Q3': [3], 'Q4': [...","{'BlockNo': [1], 'Q2': ['10 or more years'], '...",No Data,"{'BlockNo': [1], 'Q2': [2], 'Q3': [3], 'Q4': [..."


In [7]:
# Function to map education levels
def map_education_level(education):
    if 'High school' in education:
        return 'GED'
    elif 'Associate' in education or 'Bachelor' in education:
        return 'Undergraduate'
    elif 'Master' in education:
        return 'Graduate'
    else:
        return education


df_demographic = pd.DataFrame(df_survey['demographic_inventory_data'].to_list())
df_demographic = df_demographic.applymap(lambda x: x[0])  #remove list
df_demographic = pd.concat([df_survey[['userid']],df_demographic],axis=1) #add userid
df_demographic = df_demographic.rename({'Q2':'Age','Q3':'Gender','Q4':'Race','Q6':'Education',
                                       'Q7':'Smote_frequently','Q9':'Caffeine_Consumption'},axis=1) #filter and remane columns
df_demographic = df_demographic.drop(['Q5','Q8','Q10','BlockNo'],axis=1)
df_demographic['Education'] = df_demographic['Education'].apply(map_education_level)


df_demographic

,userid,Age,Gender,Race,Education,Smote_frequently,Caffeine_Consumption
0,HPLX,19,Male,Hispanic/Latino,GED,Never,Every day
1,TYDF,20,Male,Black/African-American,GED,Never,A few times per month
2,JFDE,22,Male,Asian,Undergraduate,Never,A few times per week
3,UJNH,20,Male,Black/African-American,Undergraduate,Never,Never
4,MNBW,21,Male,White/Caucasian,GED,Never,A few times per month
...,...,...,...,...,...,...,...
95,RIQW,18,Male,White/Caucasian,Undergraduate,Never,Every day
96,LKPW,20,Male,Hispanic/Latino,GED,Never,Never
97,VFTY,25,Male,Asian,Graduate,Every day,Rarely
98,POIU,24,Male,Asian,Graduate,A few times per month,Every day


In [8]:
df_gamming_exp = pd.DataFrame(df_survey['gaming_experience_data'].to_list())
df_gamming_exp = df_gamming_exp.applymap(lambda x: x[0])  #remove list
df_gamming_exp = pd.concat([df_survey[['userid']],df_gamming_exp],axis=1) #add userid
df_gamming_exp = df_gamming_exp.rename({'Q2':'Q2_How long have you played games',
                                        'Q3':'Q3_how often do you currently play video games?',
                                        'Q4':'Q4_During the average week how many hours do you currently spend playing video games?',
                                        'Q5':'Q5_When you play video games how many hours do you usually spend playing',
                                        'Q6':'Q6_How skilled do you feel you are at playing video games',
                                        'Q7':'Q7_Please rate your experience playing video games on a scale of 1 (very inexperienced) to 7 (very experienced).',
                                        'Q8':'Q8_Please rate each of the following game genres based on your preference, on a scale from 1 (Least Preferred) to 5 (Most Preferred). - First-person shooter',
                                        'Q9':'Q9_Please rate each of the following game genres based on your preference, on a scale from 1 (Least Preferred) to 5 (Most Preferred). - Realistic military',
                                        'Q10':'Q10_Please rate each of the following game genres based on your preference, on a scale from 1 (Least Preferred) to 5 (Most Preferred). - Driving or Racing',
                                        'Q11':'Q11_Please rate each of the following game genres based on your preference, on a scale from 1 (Least Preferred) to 5 (Most Preferred). - Strategy',
                                        'Q12':'Q12_Please rate each of the following game genres based on your preference, on a scale from 1 (Least Preferred) to 5 (Most Preferred). - Role-playing',
                                        'Q13':'Q13_Please rate each of the following game genres based on your preference, on a scale from 1 (Least Preferred) to 5 (Most Preferred). - Other',
                                        'Q14':'Q14_List the top 3 games you have played most frequently within the last 6 months - Top 1?',
                                        'Q15':'Q15_List the top 3 games you have played most frequently within the last 6 months - Top 2?',
                                        'Q16':'Q16_List the top 3 games you have played most frequently within the last 6 months - Top 3?',
                                        'Q17':'Q17_List any official gaming rankings that you currently have or have had in the past - Ranking',
                                        'Q18':'Q18_List any official gaming rankings that you currently have or have had in the past - Ranking',
                                        'Q19':'Q19_List any official gaming rankings that you currently have or have had in the past - Ranking',         
                                        'Q20':'Q20',
                                        'Q21':'Q21_Do you currently play in a competitive gaming league?',
                                        'Q22':'Q22_Have you ever played in a competitive gaming league?',
                                        'Q23':'Q23_Have you recorded your number of wins?',
                                        'Q24':'Q24_How many wins?',
                                        'Q25':'Q25_Total number of competitions:',
                                        'Q26':'Q26_On a scale of 1 (never) to 4 (very often) indicate how often you have played the following genres of video games. A. Stimulation',
                                        'Q27':'Q27_On a scale of 1 (never) to 4 (very often) indicate how often you have played the following genres of video games. B. Strategy',
                                        'Q28':'Q28_On a scale of 1 (never) to 4 (very often) indicate how often you have played the following genres of video games. C. Action',
                                        'Q29':'Q29_On a scale of 1 (never) to 4 (very often) indicate how often you have played the following genres of video games. D. Role Playing',
                                        'Q30':'Q30_On a scale of 1 (never) to 4 (very often) indicate how often you have played the following genres of video games. E. Unclassified',
                                       },axis=1) #filter and remane columns
df_gamming_exp = df_gamming_exp.drop('BlockNo',axis=1)





df_gamming_exp

,userid,Q2_How long have you played games,Q3_how often do you currently play video games?,Q4_During the average week how many hours do you currently spend playing video games?,Q5_When you play video games how many hours do you usually spend playing,Q6_How skilled do you feel you are at playing video games,Q7_Please rate your experience playing video games on a scale of 1 (very inexperienced) to 7 (very experienced).,"Q8_Please rate each of the following game genres based on your preference, on a scale from 1 (Least Preferred) to 5 (Most Preferred). - First-person shooter","Q9_Please rate each of the following game genres based on your preference, on a scale from 1 (Least Preferred) to 5 (Most Preferred). - Realistic military","Q10_Please rate each of the following game genres based on your preference, on a scale from 1 (Least Preferred) to 5 (Most Preferred). - Driving or Racing","Q11_Please rate each of the following game genres based on your preference, on a scale from 1 (Least Preferred) to 5 (Most Preferred). - Strategy","Q12_Please rate each of the following game genres based on your preference, on a scale from 1 (Least Preferred) to 5 (Most Preferred). - Role-playing","Q13_Please rate each of the following game genres based on your preference, on a scale from 1 (Least Preferred) to 5 (Most Preferred). - Other",Q14_List the top 3 games you have played most frequently within the last 6 months - Top 1?,Q15_List the top 3 games you have played most frequently within the last 6 months - Top 2?,Q16_List the top 3 games you have played most frequently within the last 6 months - Top 3?,Q17_List any official gaming rankings that you currently have or have had in the past - Ranking,Q18_List any official gaming rankings that you currently have or have had in the past - Ranking,Q19_List any official gaming rankings that you currently have or have had in the past - Ranking,Q20,Q21_Do you currently play in a competitive gaming league?,Q22_Have you ever played in a competitive gaming league?,Q23_Have you recorded your number of wins?,Q24_How many wins?,Q25_Total number of competitions:,Q26_On a scale of 1 (never) to 4 (very often) indicate how often you have played the following genres of video games. A. Stimulation,Q27_On a scale of 1 (never) to 4 (very often) indicate how often you have played the following genres of video games. B. Strategy,Q28_On a scale of 1 (never) to 4 (very often) indicate how often you have played the following genres of video games. C. Action,Q29_On a scale of 1 (never) to 4 (very often) indicate how often you have played the following genres of video games. D. Role Playing,Q30_On a scale of 1 (never) to 4 (very often) indicate how often you have played the following genres of video games. E. Unclassified
0,HPLX,5-10 years,Almost every day,8-10 hours,Less than 2 hours,Moderately skilled,6,5,2,4,3,4,3,Rhythm,Halo Infinite,Minecraft,Horizon: zero dawn,,,,No,No,No,,,1,2,4,4,3
1,TYDF,10 or more years,1-3 times a month,Less than 2 hours,8-10 hours,Not very skilled,6,4,3,2,4,3,4,Fighting Games,Super Smash brothers ultimate,Minecraft,Roblox,5th UMBC SMASH RUMBLE,,,No,No,No,,,2,1,1,1,4
2,JFDE,10 or more years,1-3 times a month,Less than 2 hours,Less than 2 hours,Moderately skilled,5,5,1,3,3,2,5,FIFA,Valorant,Call of Duty,Fortnite,,,,No,No,Yes,600,0,3,1,4,1,3
3,UJNH,10 or more years,Once or twice a week,2-5 hours,2-5 hours,Moderately skilled,7,2,3,1,5,5,4,Third person action,Kenshi,FrostPunk,OtherCide,,,,No,No,No,,,1,4,2,4,1
4,MNBW,10 or more years,Almost every day,8-10 hours,2-5 hours,Very skilled,7,5,3,2,4,1,1,,Valorant,Overwatch 2,Tetris,Radiant in Valorant (~#400 on leaderboard),Plat 1 in Overwatch,B+ in Tetr.io,No,Yes,No,,,2,1,4,1,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,RIQW,10 or more years,Almost every day,10-15 hours,2-5 hours,Moderately skilled,6,5,3,3,1,4,2,SANDBOX,LIMBUS COMPANY,ULTRAKILL,COUNTERSTRIKE 2,,,,No,No,No,,,1

In [9]:
df_merge = pd.merge(df_demographic,df_gamming_exp,on='userid')
df_merge = df_merge.rename({'userid':'subject_id'},axis=1)
df_merge

,subject_id,Age,Gender,Race,Education,Smote_frequently,Caffeine_Consumption,Q2_How long have you played games,Q3_how often do you currently play video games?,Q4_During the average week how many hours do you currently spend playing video games?,Q5_When you play video games how many hours do you usually spend playing,Q6_How skilled do you feel you are at playing video games,Q7_Please rate your experience playing video games on a scale of 1 (very inexperienced) to 7 (very experienced).,"Q8_Please rate each of the following game genres based on your preference, on a scale from 1 (Least Preferred) to 5 (Most Preferred). - First-person shooter","Q9_Please rate each of the following game genres based on your preference, on a scale from 1 (Least Preferred) to 5 (Most Preferred). - Realistic military","Q10_Please rate each of the following game genres based on your preference, on a scale from 1 (Least Preferred) to 5 (Most Preferred). - Driving or Racing","Q11_Please rate each of the following game genres based on your preference, on a scale from 1 (Least Preferred) to 5 (Most Preferred). - Strategy","Q12_Please rate each of the following game genres based on your preference, on a scale from 1 (Least Preferred) to 5 (Most Preferred). - Role-playing","Q13_Please rate each of the following game genres based on your preference, on a scale from 1 (Least Preferred) to 5 (Most Preferred). - Other",Q14_List the top 3 games you have played most frequently within the last 6 months - Top 1?,Q15_List the top 3 games you have played most frequently within the last 6 months - Top 2?,Q16_List the top 3 games you have played most frequently within the last 6 months - Top 3?,Q17_List any official gaming rankings that you currently have or have had in the past - Ranking,Q18_List any official gaming rankings that you currently have or have had in the past - Ranking,Q19_List any official gaming rankings that you currently have or have had in the past - Ranking,Q20,Q21_Do you currently play in a competitive gaming league?,Q22_Have you ever played in a competitive gaming league?,Q23_Have you recorded your number of wins?,Q24_How many wins?,Q25_Total number of competitions:,Q26_On a scale of 1 (never) to 4 (very often) indicate how often you have played the following genres of video games. A. Stimulation,Q27_On a scale of 1 (never) to 4 (very often) indicate how often you have played the following genres of video games. B. Strategy,Q28_On a scale of 1 (never) to 4 (very often) indicate how often you have played the following genres of video games. C. Action,Q29_On a scale of 1 (never) to 4 (very often) indicate how often you have played the following genres of video games. D. Role Playing,Q30_On a scale of 1 (never) to 4 (very often) indicate how often you have played the following genres of video games. E. Unclassified
0,HPLX,19,Male,Hispanic/Latino,GED,Never,Every day,5-10 years,Almost every day,8-10 hours,Less than 2 hours,Moderately skilled,6,5,2,4,3,4,3,Rhythm,Halo Infinite,Minecraft,Horizon: zero dawn,,,,No,No,No,,,1,2,4,4,3
1,TYDF,20,Male,Black/African-American,GED,Never,A few times per month,10 or more years,1-3 times a month,Less than 2 hours,8-10 hours,Not very skilled,6,4,3,2,4,3,4,Fighting Games,Super Smash brothers ultimate,Minecraft,Roblox,5th UMBC SMASH RUMBLE,,,No,No,No,,,2,1,1,1,4
2,JFDE,22,Male,Asian,Undergraduate,Never,A few times per week,10 or more years,1-3 times a month,Less than 2 hours,Less than 2 hours,Moderately skilled,5,5,1,3,3,2,5,FIFA,Valorant,Call of Duty,Fortnite,,,,No,No,Yes,600,0,3,1,4,1,3
3,UJNH,20,Male,Black/African-American,Undergraduate,Never,Never,10 or more years,Once or twice a week,2-5 hours,2-5 hours,Moderately skilled,7,2,3,1,5,5,4,Third person action,Kenshi,FrostPunk,OtherCide,,,,No,No,No,,,1,4,2,4,1
4,MNBW,21,Male,White/Caucasian,GED,Never,A few times per month,10 or more years,Almost every day,8-10 hours,2-5 hours,Very skilled,7,5,3,2,4,1,1,,Valorant,Overwatch 2,Tetris,Radiant in Valorant (~#400 on leaderboard

In [10]:
df_merge_Descartes = pd.merge(df,df_merge,on='subject_id',how='inner')
df_merge_Descartes

,subject_id,block_type,q2,q3,q4,q6,q9,q10,tab_key,session_id,total_damage_taken,total_shot_all,total_kill,total_death,total_shot_hit,total_engage,rocket_shot_hit,gun_shot_hit,self_kill,gender,race,trait_score,state_score,trust_score,mode_cluster,Age,Gender,Race,Education,Smote_frequently,Caffeine_Consumption,Q2_How long have you played games,Q3_how often do you currently play video games?,Q4_During the average week how many hours do you currently spend playing video games?,Q5_When you play video games how many hours do you usually spend playing,Q6_How skilled do you feel you are at playing video games,Q7_Please rate your experience playing video games on a scale of 1 (very inexperienced) to 7 (very experienced).,"Q8_Please rate each of the following game genres based on your preference, on a scale from 1 (Least Preferred) to 5 (Most Preferred). - First-person shooter","Q9_Please rate each of the following game genres based on your preference, on a scale from 1 (Least Preferred) to 5 (Most Preferred). - Realistic military","Q10_Please rate each of the following game genres based on your preference, on a scale from 1 (Least Preferred) to 5 (Most Preferred). - Driving or Racing","Q11_Please rate each of the following game genres based on your preference, on a scale from 1 (Least Preferred) to 5 (Most Preferred). - Strategy","Q12_Please rate each of the following game genres based on your preference, on a scale from 1 (Least Preferred) to 5 (Most Preferred). - Role-playing","Q13_Please rate each of the following game genres based on your preference, on a scale from 1 (Least Preferred) to 5 (Most Preferred). - Other",Q14_List the top 3 games you have played most frequently within the last 6 months - Top 1?,Q15_List the top 3 games you have played most frequently within the last 6 months - Top 2?,Q16_List the top 3 games you have played most frequently within the last 6 months - Top 3?,Q17_List any official gaming rankings that you currently have or have had in the past - Ranking,Q18_List any official gaming rankings that you currently have or have had in the past - Ranking,Q19_List any official gaming rankings that you currently have or have had in the past - Ranking,Q20,Q21_Do you currently play in a competitive gaming league?,Q22_Have you ever played in a competitive gaming league?,Q23_Have you recorded your number of wins?,Q24_How many wins?,Q25_Total number of competitions:,Q26_On a scale of 1 (never) to 4 (very often) indicate how often you have played the following genres of video games. A. Stimulation,Q27_On a scale of 1 (never) to 4 (very often) indicate how often you have played the following genres of video games. B. Strategy,Q28_On a scale of 1 (never) to 4 (very often) indicate how often you have played the following genres of video games. C. Action,Q29_On a scale of 1 (never) to 4 (very often) indicate how often you have played the following genres of video games. D. Role Playing,Q30_On a scale of 1 (never) to 4 (very often) indicate how often you have played the following genres of video games. E. Unclassified
0,BVCX,block_01,4,2,3,3,38,32,0,1263e8ef-4b1f-9a88-a07b-85971cac44ab,729.192802,141,2,11,14,17,0,14,0,Male,Prefer not to say,44.0,48.0,94,3,27,Male,Prefer not to say,Graduate,Never,A few times per month,6 months,Less than once a month,Less than 2 hours,2-5 hours,Not very skilled,4,4,4,5,3,2,4,Coding Games,GTA 5,Call of Duty,Counter Strike Go,,,,No,No,No,,,3,2,3,1,1
1,BVCX,block_02,4,4,3,4,21,18,0,6ee85647-4297-186c-24fa-c4970441459c,1583.916572,99,3,21,30,18,0,30,0,Male,Prefer not to say,44.0,48.0,94,3,27,Male,Prefer not to say,Graduate,Never,A few times per month,6 months,Less than once a month,Less than 2 hours,2-5 hours,Not very skilled,4,4,4,5,3,2,4,Coding Games,GTA 5,Call of Duty,Counter Strike Go,,,,No,No,No,,,3,2,3,1,1
2,BVCX,block_03,4,2,3,4,15,23,0,b8103d26-48b2-edb7-9cf6-febff2e0b27b,1035.726212,173,2,16,35,29,0,35,0,Male,Prefer not to say,44.0,48.0,94,3,27,Male,Prefer not to say,Graduate,Never,A few t

In [13]:
conn.close()

# Shooting game survey

In [25]:
DB_NAME = "tack"

conn = psycopg2.connect(
    host="127.0.0.1",
    port="5432",
    database=DB_NAME,
    user="postgres",
    password="postgres"
)



In [26]:
# Execute the SQL query
cursor = conn.cursor()
sql_query = "SELECT * FROM tack.shooting_survey_data"
cursor.execute(sql_query)

# Fetch all the results
columns = [desc[0] for desc in cursor.description]
data = cursor.fetchall()
df_survey_sg = pd.DataFrame(data, columns=columns)

cursor.close()
df_survey_sg

,userid,game_familiarity,game_questions
0,QPOI,"{'BlockNo': [1], 'Q2': ['This person is a stra...","{'BlockNo': [1, 2, 3, 4], 'Q2': ['Komodo', 'Ko..."
1,DFZX,"{'BlockNo': [1], 'Q2': ['This person is a stra...","{'BlockNo': [1, 2, 3, 4], 'Q2': ['Gila', 'Gila..."
2,QWER,"{'BlockNo': [1], 'Q2': ['This person is a stra...","{'BlockNo': [1, 2, 3, 4], 'Q2': ['Gila', 'Gila..."
3,MJUY,"{'BlockNo': [1], 'Q2': ['This person is a stra...","{'BlockNo': [1, 2, 3, 4], 'Q2': ['Panda', 'Pan..."
4,UYGH,"{'BlockNo': [1], 'Q2': ['This person is a stra...","{'BlockNo': [1, 2, 3, 4], 'Q2': ['Komodo', 'Ko..."
5,MJKL,"{'BlockNo': [1], 'Q2': ['This person is a stra...","{'BlockNo': [1, 2, 3, 4], 'Q2': ['Thylacine', ..."
6,DKLQ,"{'BlockNo': [1], 'Q2': ['This person is a stra...","{'BlockNo': [1, 2, 3, 4], 'Q2': ['Komodo', 'Ko..."
7,LKPW,"{'BlockNo': [1], 'Q2': ['This person is a stra...","{'BlockNo': [1, 2, 3, 4], 'Q2': ['Thylacine', ..."
8,FTLD,"{'BlockNo': [1], 'Q2': ['This person is a frie...","{'BlockNo': [1, 2, 3, 4], 'Q2': ['Gila', 'Gila..."
9,HPLX,"{'BlockNo': [1], 'Q2': ['This person is a stra...","{'BlockNo': [1, 2, 3, 4], 'Q2': ['Thylacine', ..."


In [32]:
df_game_fami = pd.DataFrame(df_survey_sg['game_familiarity'].to_list())
df_game_fami = df_game_fami.applymap(lambda x: x[0])  #remove list
df_game_fami = pd.concat([df_survey_sg[['userid']],df_game_fami],axis=1) #add userid
df_game_fami = df_game_fami.rename({'Q2':'Q2_What is the relationship between you and your teammate?',
                                    'Q3':'Q3_What is the relationship between you and opponent 1?',
                                    'Q4':'Q4_What is the relationship between you and opponent 2?',
                                    'userid':'subject_id',
                                   },axis=1) #filter and remane columns
df_game_fami = df_game_fami.drop('BlockNo',axis=1)

df_game_fami

,subject_id,Q2_What is the relationship between you and your teammate?,Q3_What is the relationship between you and opponent 1?,Q4_What is the relationship between you and opponent 2?
0,QPOI,This person is a stranger. I am meeting them f...,This person is a friend. I know this person we...,This person is a stranger. I am meeting them f...
1,DFZX,This person is a stranger. I am meeting them f...,This person is a stranger. I am meeting them f...,This person is a stranger. I am meeting them f...
2,QWER,This person is a stranger. I am meeting them f...,This person is a stranger. I am meeting them f...,This person is a stranger. I am meeting them f...
3,MJUY,This person is a stranger. I am meeting them f...,This person is a stranger. I am meeting them f...,This person is a stranger. I am meeting them f...
4,UYGH,This person is a stranger. I am meeting them f...,This person is a stranger. I am meeting them f...,This person is a stranger. I am meeting them f...
5,MJKL,This person is a stranger. I am meeting them f...,This person is a stranger. I am meeting them f...,This person is a stranger. I am meeting them f...
6,DKLQ,This person is a stranger. I am meeting them f...,This person is a stranger. I am meeting them f...,This person is a friend. I know this person we...
7,LKPW,This person is a stranger. I am meeting them f...,This person is a stranger. I am meeting them f...,This person is a stranger. I am meeting them f...
8,FTLD,This person is a friend. I know this person we...,This person is a friend. I know this person we...,This person is a friend. I know this person we...
9,HPLX,This person is a stranger. I am meeting them f...,This person is a stranger. I am meeting them f...,This person is a stranger. I am meeting them f...


In [33]:
df_merge_Descartes_2 = pd.merge(df_merge_Descartes,df_game_fami,on='subject_id',how='left')
df_merge_Descartes_2

,subject_id,block_type,q2,q3,q4,q6,q9,q10,tab_key,session_id,total_damage_taken,total_shot_all,total_kill,total_death,total_shot_hit,total_engage,rocket_shot_hit,gun_shot_hit,self_kill,gender,race,trait_score,state_score,trust_score,mode_cluster,Age,Gender,Race,Education,Smote_frequently,Caffeine_Consumption,Q2_How long have you played games,Q3_how often do you currently play video games?,Q4_During the average week how many hours do you currently spend playing video games?,Q5_When you play video games how many hours do you usually spend playing,Q6_How skilled do you feel you are at playing video games,Q7_Please rate your experience playing video games on a scale of 1 (very inexperienced) to 7 (very experienced).,"Q8_Please rate each of the following game genres based on your preference, on a scale from 1 (Least Preferred) to 5 (Most Preferred). - First-person shooter","Q9_Please rate each of the following game genres based on your preference, on a scale from 1 (Least Preferred) to 5 (Most Preferred). - Realistic military","Q10_Please rate each of the following game genres based on your preference, on a scale from 1 (Least Preferred) to 5 (Most Preferred). - Driving or Racing","Q11_Please rate each of the following game genres based on your preference, on a scale from 1 (Least Preferred) to 5 (Most Preferred). - Strategy","Q12_Please rate each of the following game genres based on your preference, on a scale from 1 (Least Preferred) to 5 (Most Preferred). - Role-playing","Q13_Please rate each of the following game genres based on your preference, on a scale from 1 (Least Preferred) to 5 (Most Preferred). - Other",Q14_List the top 3 games you have played most frequently within the last 6 months - Top 1?,Q15_List the top 3 games you have played most frequently within the last 6 months - Top 2?,Q16_List the top 3 games you have played most frequently within the last 6 months - Top 3?,Q17_List any official gaming rankings that you currently have or have had in the past - Ranking,Q18_List any official gaming rankings that you currently have or have had in the past - Ranking,Q19_List any official gaming rankings that you currently have or have had in the past - Ranking,Q20,Q21_Do you currently play in a competitive gaming league?,Q22_Have you ever played in a competitive gaming league?,Q23_Have you recorded your number of wins?,Q24_How many wins?,Q25_Total number of competitions:,Q26_On a scale of 1 (never) to 4 (very often) indicate how often you have played the following genres of video games. A. Stimulation,Q27_On a scale of 1 (never) to 4 (very often) indicate how often you have played the following genres of video games. B. Strategy,Q28_On a scale of 1 (never) to 4 (very often) indicate how often you have played the following genres of video games. C. Action,Q29_On a scale of 1 (never) to 4 (very often) indicate how often you have played the following genres of video games. D. Role Playing,Q30_On a scale of 1 (never) to 4 (very often) indicate how often you have played the following genres of video games. E. Unclassified,Q2_What is the relationship between you and your teammate?,Q3_What is the relationship between you and opponent 1?,Q4_What is the relationship between you and opponent 2?
0,BVCX,block_01,4,2,3,3,38,32,0,1263e8ef-4b1f-9a88-a07b-85971cac44ab,729.192802,141,2,11,14,17,0,14,0,Male,Prefer not to say,44.0,48.0,94,3,27,Male,Prefer not to say,Graduate,Never,A few times per month,6 months,Less than once a month,Less than 2 hours,2-5 hours,Not very skilled,4,4,4,5,3,2,4,Coding Games,GTA 5,Call of Duty,Counter Strike Go,,,,No,No,No,,,3,2,3,1,1,This person is a friend. I know this person we...,This person is a stranger. I am meeting them f...,This person is a stranger. I am meeting them f...
1,BVCX,block_02,4,4,3,4,21,18,0,6ee85647-4297-186c-24fa-c4970441459c,1583.916572,99,3,21,30,18,0,30,0,Male,Prefer not to say,44.0,48.0,94,3,27,Male,Prefer not to say,Graduate,Never,A few times per month,6 months,Less than once a month,L

# Save

In [56]:
df_merge.to_csv("data_100_subjectid_demographic_gaming_exp.csv",index=False)
df_merge_Descartes_2.to_csv("Descartes Data with demographic and gaming exp.csv",index=False)

In [24]:
conn.close()